In [ ]:
from vnpy.app.cta_strategy.backtesting import BacktestingEngine, OptimizationSetting
from fixed_hedge_strategy import FixedHedgeStrategy
from dynamic_hedge_strategy import DynamicHedgeStrategy
from datetime import datetime

In [ ]:
# 固定区间策略回测
params = {
    'hedge_range': 0.05,
    'hedge_multiple': 1.0
}

engine = BacktestingEngine()
engine.set_parameters(
    vt_symbol="510050.SSE",
    interval="1m",
    start=datetime(2019, 1, 1),
    end=datetime(2020, 4, 1),
    rate=1/10000,
    slippage=0.002,
    size=10000,
    pricetick=0.001,
    capital=1_000_000,
)
engine.add_strategy(FixedHedgeStrategy, params)

In [ ]:
# 动态区间策略回测
params = {
    'hedge_range_percent': 30,
    'hedge_multiple_percent': 50
}

engine = BacktestingEngine()
engine.set_parameters(
    vt_symbol="510050.SSE",
    interval="1m",
    start=datetime(2019, 1, 1),
    end=datetime(2020, 4, 1),
    rate=1/10000,
    slippage=0.002,
    size=10000,
    pricetick=0.001,
    capital=1_000_000,
)
engine.add_strategy(DynamicHedgeStrategy, params)

In [ ]:
engine.load_data()
engine.run_backtesting()
df = engine.calculate_result()
engine.calculate_statistics()
engine.show_chart()

In [ ]:
import pickle
import pandas as pd
from utility import dt_to_str

with open('result/result_2015-01-01.dat', 'rb') as f:
    data = pickle.load(f)

In [ ]:
start = ''
result_dict_list = []
for tp in data:
    tp[2]['setting'] = tp[0]
    start = dt_to_str(tp[2]['start_date'])
    result_dict_list.append(tp[2])
print(start)
# result_dict_list
df = pd.DataFrame(result_dict_list)
df.to_csv(f'result/result_{start}.csv', index=False)